<a href="https://colab.research.google.com/github/ayushgupta9198/English_Language_translation/blob/master/17_02fairseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/facebook/fairseq

/content/drive/My Drive/facebook/fairseq


In [0]:
ls

build/                                           fairseq_logo.png
checkpoints/                                     generate.py
CODE_OF_CONDUCT.md                               hubconf.py
CONTRIBUTING.md                                  interactive.py
custom.en-zh.en.bpe16k/                          iwslt17.de_fr.en.bpe16k/
custom.en-zh.en.bpe16k-20200219T094512Z-001.zip  iwslt17_orig/
custom_orig/                                     LICENSE
custom_orig-20200219T094529Z-001.zip             preprocess.py
Data/                                            pytorch_fairseq_master/
data-bin/                                        README.md
docs/                                            score.py
eval_lm.py                                       scripts/
examples/                                        setup.py
fairseq/                                         tests/
fairseq_cli/                                     train.py
fairseq.egg-info/                                validate.py
fairseq.gif

In [0]:
pip install sentencepiece

     |████████████████████████████████| 1.0MB 2.8MB/s 


In [29]:
pwd

'/content/drive/My Drive/facebook/fairseq'

In [0]:
%%bash

SRCS=(
  "en"
)
TGT=zh

ROOT=$(dirname "$0")
SCRIPTS=./scripts
SPM_TRAIN=$SCRIPTS/spm_train.py
SPM_ENCODE=$SCRIPTS/spm_encode.py

BPESIZE=16384
ORIG=$ROOT/OrignalData
DATA=$ROOT/OrignalData.en-zh.bpe16k
mkdir -p "$ORIG" "$DATA"

In [0]:
%%bash
TRAIN_MINLEN=1  # remove sentences with <1 BPE token
TRAIN_MAXLEN=250  # remove sentences with >250 BPE tokens



In [0]:
%%bash

echo "pre-processing train data... done"

for LANG in "en" "zh"; do
    cat "/content/drive/My Drive/facebook/fairseq/OrignalData/test_en-zh."${LANG} \
        | grep -v '<url>' \
        | grep -v '<talkid>' \
        | grep -v '<keywords>' \
        | grep -v '<speaker>' \
        | grep -v '<reviewer' \
        | grep -v '<translator' \
        | grep -v '<doc' \
        | grep -v '</doc>' \
        | sed -e 's/<title>//g' \
        | sed -e 's/<\/title>//g' \
        | sed -e 's/<description>//g' \
        | sed -e 's/<\/description>//g' \
        | sed 's/^\s*//g' \
        | sed 's/\s*$//g' \
        > "/content/drive/My Drive/facebook/fairseq/OrignalData.en-zh.bpe16k/test_en-zh."${LANG}
done

pre-processing train data... done


In [46]:
pip install sentencepiece

  Using cached https://files.pythonhosted.org/packages/74/f4/2d5214cbf13d06e7cb2c20d84115ca25b53ea76fa1f0ade0e3c9749de214/sentencepiece-0.1.85-cp36-cp36m-manylinux1_x86_64.whl


In [48]:
%%bash
# learn BPE with sentencepiece
TRAIN_FILES=$(for SRC in "${SRCS[@]}"; do \
echo $DATA/train_${SRC}-${TGT}.${SRC}; \
echo $DATA/train_${SRC}-${TGT}.${TGT}; done | tr "\n" ",")
echo "learning joint BPE over ${TRAIN_FILES}..."
python /content/drive/My\ Drive/facebook/fairseq/scripts/spm_train.py \
    --input=$TRAIN_FILES \
    --model_prefix=$DATA/sentencepiece.bpe \
    --vocab_size=$BPESIZE \
    --character_coverage=1.0 \
    --model_type=bpe
    


learning joint BPE over ...


sentencepiece_trainer.cc(116) LOG(INFO) Running command: --input= --model_prefix=/sentencepiece.bpe --vocab_size= --character_coverage=1.0 --model_type=bpe
Traceback (most recent call last):
  File "/content/drive/My Drive/facebook/fairseq/scripts/spm_train.py", line 16, in <module>
    spm.SentencePieceTrainer.Train(" ".join(sys.argv[1:]))
SyntaxError: Invalid argument: cannot parse "" as int32.


In [99]:
%%bash
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

SRCS=(
  "en"
)
TGT=zh

ROOT=$(dirname "$0")
SCRIPTS=./scripts
SPM_TRAIN=$SCRIPTS/spm_train.py
SPM_ENCODE=$SCRIPTS/spm_encode.py

BPESIZE=16384
ORIG=$ROOT/OrignalData
DATA=$ROOT/OrignalData.en-zh.bpe16k
# mkdir -p "$ORIG" "$DATA"

TRAIN_MINLEN=1  # remove sentences with <1 BPE token
TRAIN_MAXLEN=250  # remove sentences with >250 BPE tokens



# learn BPE with sentencepiece
TRAIN_FILES=$(for SRC in "${SRCS[@]}"; do \
echo $DATA/train_${SRC}-${TGT}.${SRC}; \
echo $DATA/train_${SRC}-${TGT}.${TGT}; done | tr "\n" ",")
echo "learning joint BPE over ${TRAIN_FILES}..."
python "$SPM_TRAIN" \
    --input=$TRAIN_FILES \
    --model_prefix=$DATA/sentencepiece.bpe \
    --vocab_size=$BPESIZE \
    --character_coverage=1.0 \
    --model_type=bpe

# encode train/valid/test
echo "encoding train/valid with learned BPE..."
for SRC in "${SRCS[@]}"; do
    for LANG in "$SRC" "$TGT"; do
        python "$SPM_ENCODE" \
            --model "$DATA/sentencepiece.bpe.model" \
            --output_format=piece \
            --inputs $DATA/train_${SRC}-${TGT}.${SRC} $DATA/train_${SRC}-${TGT}.${TGT} \
            --outputs $DATA/train.bpe.${SRC}-${TGT}.${SRC} $DATA/train.bpe.${SRC}-${TGT}.${TGT} \
            --min-len $TRAIN_MINLEN --max-len $TRAIN_MAXLEN
        python "$SPM_ENCODE" \
            --model "$DATA/sentencepiece.bpe.model" \
            --output_format=piece \
             --inputs $DATA/valid_${SRC}-${TGT}.${SRC} $DATA/valid_${SRC}-${TGT}.${TGT} \
            --outputs $DATA/valid.bpe.${SRC}-${TGT}.${SRC} $DATA/valid.bpe.${SRC}-${TGT}.${TGT}
        python "$SPM_ENCODE" \
            --model "$DATA/sentencepiece.bpe.model" \
            --output_format=piece \
            --inputs $DATA/test_${SRC}-${TGT}.${SRC} $DATA/test_${SRC}-${TGT}.${TGT} \
            --outputs $DATA/test.bpe.${SRC}-${TGT}.${SRC} $DATA/test.bpe.${SRC}-${TGT}.${TGT}
    done
done

learning joint BPE over ./OrignalData.en-zh.bpe16k/train_en-zh.en,./OrignalData.en-zh.bpe16k/train_en-zh.zh,...
encoding train/valid with learned BPE...


sentencepiece_trainer.cc(116) LOG(INFO) Running command: --input=./OrignalData.en-zh.bpe16k/train_en-zh.en,./OrignalData.en-zh.bpe16k/train_en-zh.zh, --model_prefix=./OrignalData.en-zh.bpe16k/sentencepiece.bpe --vocab_size=16384 --character_coverage=1.0 --model_type=bpe
sentencepiece_trainer.cc(49) LOG(INFO) Starts training with : 
TrainerSpec {
  input: ./OrignalData.en-zh.bpe16k/train_en-zh.en
  input: ./OrignalData.en-zh.bpe16k/train_en-zh.zh
  input_format: 
  model_prefix: ./OrignalData.en-zh.bpe16k/sentencepiece.bpe
  model_type: BPE
  vocab_size: 16384
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  treat_whitespace_as_suffix: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id:

In [92]:
!pip install fairseq

     |████████████████████████████████| 307kB 3.3MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2015618 sha256=83be7fc2b894926febe7e5f179bc1721f3fe57917c9465f7528a0302c4cbc902
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq


In [0]:
# Binarize the dataset
!fairseq-preprocess \
    --source-lang en \
    --target-lang zh \
    --trainpref /content/drive/My\ Drive/facebook/fairseq/OrignalData.en-zh.bpe16k/train.bpe.en-zh\
    --validpref /content/drive/My\ Drive/facebook/fairseq/OrignalData.en-zh.bpe16k/valid.bpe.en-zh \
    --testpref /content/drive/My\ Drive/facebook/fairseq/OrignalData.en-zh.bpe16k/test.bpe.en-zh \
    --destdir data-bin/custom_en_zh \
    --thresholdtgt 0 --thresholdsrc 0 \
    --workers 1

Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/custom_en_zh', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='en', srcdict=None, target_lang='zh', task='translation', tensorboard_logdir='', testpref='/content/drive/My Drive/facebook/fairseq/OrignalData.en-zh.bpe16k/test.bpe.en-zh', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='/content/drive/My Drive/facebook/fairseq/OrignalData.en-zh.bpe16k/train.bpe.en-zh', user_dir=None, validpref='/content/drive/My Drive/facebook/fairseq/OrignalData.en-zh.bpe16k/valid.bpe.en-zh', workers=1)
| [en] Dict